In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from constants import *
from train_collections import *
# read the pickle file and print the data
def read_pickle_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    # print(data)
    return data

harakat = read_pickle_file('./Delivery/diacritic2id.pickle')
print('-----------------------------')
read_pickle_file('./Delivery/diacritics.pickle')
print('-----------------------------')
lettes = read_pickle_file('./Delivery/arabic_letters.pickle')

# print(len(lettes))
# print(type(lettes))
# print(ARABIC_LETTERS)
# print(len(ARABIC_LETTERS))
print(harakat2id)


-----------------------------
-----------------------------
{'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}


In [6]:
import re

text = "BBARRCBAB"
re.split(r'[AC]', text)

['BB', 'RR', 'B', 'B']